# How much is your Fiat 500 worth?
## Skills: Python, Web scraping, Geopy, Regression

Download the Jupyter Notebook from [GitHub](https://nicomoriuala.github.io)

<a id='content' />

## Contents

1. [Introduction](#intro)
1. [Web Scraping](#scraping)
2. [Data Cleansing](#cleansing)
3. [Mapping](#mapping)
4. [A definir](#volatility)

<a id='intro' />

## Introduction
[(back to top)](#content)

In [2]:
#Importing packages
import pandas as pd
import numpy as np
import re
import unidecode

from tqdm.notebook import tqdm
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

<a id='scraping' />

## Web Scraping
[(back to top)](#content)

In [36]:
#Load webdriver
driver = webdriver.Chrome('/Users/nicolamori/Library/Application Support/binman_chromedriver/mac64/85.0.4183.38/chromedriver')

#Open page
driver.get('https://www.automobile.it/fiat-500-usate?anno_fino_a=2020&immatricolazione=2007')
time.sleep(4)

#Close cookies popup
driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
time.sleep(2)

In [37]:
#Empty list and df
temp = []
df_ = pd.DataFrame(columns=['Title', 'Province','Condition', 'Month_Year', 'Kms', 'Fuel', 'Transmission', 'Price'])

#Number of pages
npages = int(driver.find_element_by_xpath('//*[@id="__next"]/main/div[2]/div/div[4]/div[2]/div[3]/div[1]/button[4]').text)

for h in tqdm(range(npages), position = 0, desc='1st loop'):
    

    #Title and Province
    titles=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879"]')
    for title in titles: temp.append(title.text)
    
    del temp[1::3]
    temp = [temp[x:x+2] for x in range(0, len(temp),2)]
    
    #Dataframe and clear list
    part_1 = pd.DataFrame(temp, columns=['Title', 'Province'])  
    temp.clear()
    
    
    #Specs
    specs=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879 Card__InfoTag font-base"]')
    for spec in specs: temp.append(spec.text)
    
    while 'Unico proprietario' in temp: temp.remove('Unico proprietario')
    while '2 Proprietari' in temp: temp.remove('2 Proprietari')
    while '3 Proprietari' in temp: temp.remove('3 Proprietari')
    while '4 Proprietari' in temp: temp.remove('4 Proprietari')
        
    size = len(temp) 
    idx_list = [idx for idx, val in enumerate(temp) if val == 'Usato'] 
    temp = [temp[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))] 
    del temp[0:0 + 1]
    
    #Dataframe and clear list
    part_2 = pd.DataFrame(temp, columns=['Condition', 'Month_Year', 'Kms', 'Fuel', 'Transmission'])
    temp.clear()
    
    
    #Price
    prices=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879 Card__InfoPrice font-medium"]')
    for price in prices: temp.append(price.text)
        
    #Dataframe and clear list
    part_3 = pd.DataFrame(temp, columns=['Price'])
    temp.clear()
    
    
    #Combine parts and append to dataframe
    df = pd.concat([part_1.reset_index(drop=True), part_2.reset_index(drop=True), part_3.reset_index(drop=True)], axis=1)
    df_ = df_.append(df, ignore_index = True)
    
    
    #Switch to next page or close
    if h < max(range(npages)):
        driver.find_element_by_xpath('//*[@class="jsx-3190051910 font-base xxsmall primary-orange styled"]').click()
        time.sleep(3)
    else: driver.close()
    
    
#df_.to_csv('fiat500_prices.tsv', sep='\t', index=False)


CPU times: user 26.1 s, sys: 2.32 s, total: 28.4 s
Wall time: 20min 55s


<a id='cleansing' />

## Data Cleansing
[(back to top)](#content)

In [22]:
fiat_500=pd.read_csv('fiat500_prices.tsv', sep='\t')

fiat_500.head()

,Title,Province,Condition,Month_Year,Kms,Fuel,Transmission,Owners,Price
0,Fiat 500 abarth fiat 500e elettrica abarth pri...,Verona (VR),Usato,Dicembre 2015,29.233 km,Elettrico,Automatico,NaN,€ 9.991
1,FIAT 500,Giugliano in Campania (NA),Usato,Luglio 2013,51.000 km,Benzina - Euro 5,Manuale,NaN,€ 4.990
2,Fiat 500 1.2 69cv Collezione Tetto,Ciampino (RM),Usato,Gennaio 2019,31.412 km,Benzina - Euro 6,Manuale,NaN,€ 9.980
3,FIAT 500 1.2 Lounge,Moncalieri (TO),Usato,Aprile 2019,14.918 km,Benzina - Euro 6,Manuale,NaN,€ 9.900
4,FIAT 500 1.2 Lounge,Torino (TO),Usato,Maggio 2019,18.539 km,Benzina - Euro 6,Manuale,NaN,€ 9.700


In [23]:
conditions = [fiat_500['Title'].str.contains('Lounge', case=False), 
              fiat_500['Title'].str.contains('Pop', case=False), 
              fiat_500['Title'].str.contains('Collezione', case=False), 
              fiat_500['Title'].str.contains('Star', case=False), 
              fiat_500['Title'].str.contains('Sport', case=False), 
              fiat_500['Title'].str.contains('S ', case=False), 
              fiat_500['Title'].str.contains(' S', case=False),  
              fiat_500['Title'].str.contains('"S"', case=False)]

choices = ['Lounge', 'Pop', 'Collezione', 'Star', 'Sport', 'Sport', 'Sport', 'Sport']

fiat_500['Version'] = np.select(conditions, choices, default='None')

In [24]:
fiat_500['Kms'] = ([i.replace(' km','') for i in fiat_500['Kms']])
fiat_500['Kms'] = pd.to_numeric(fiat_500['Kms'])*1000

fiat_500['Price'] = ([i.replace('€ ','') for i in fiat_500['Price']])
fiat_500['Price'] = pd.to_numeric(fiat_500['Price'])*1000

fiat_500['Fuel'] = ([i.replace(' - Euro ','') for i in fiat_500['Fuel']])
fiat_500['Fuel'] = fiat_500['Fuel'].str.replace('\d+', '')

fiat_500['Year'] = fiat_500['Month_Year'].str.strip().str[-4:]

In [25]:
provinces = pd.read_csv('it_provinces.csv', sep=';',  keep_default_na=False)

fiat_500['Province'] = fiat_500['Province'].str.strip().str[-3:-1]
fiat_500 = fiat_500.merge(provinces, on='Province', how='left')

In [26]:
del fiat_500['Month_Year']
del fiat_500['Owners']
del fiat_500['Condition']
del fiat_500['Province']
del fiat_500['Name']

fiat_500 = fiat_500[~fiat_500['Title'].str.contains('Abarth', case=False)]
fiat_500 = fiat_500[~fiat_500['Title'].str.contains('C ', case=False)]

fiat_500.head()

,Title,Kms,Fuel,Transmission,Price,Version,Year,Region
1,FIAT 500,51000.0,Benzina,Manuale,4990.0,None,2013,Campania
2,Fiat 500 1.2 69cv Collezione Tetto,31412.0,Benzina,Manuale,9980.0,Collezione,2019,Lazio
3,FIAT 500 1.2 Lounge,14918.0,Benzina,Manuale,9900.0,Lounge,2019,Piemonte
4,FIAT 500 1.2 Lounge,18539.0,Benzina,Manuale,9700.0,Lounge,2019,Piemonte
5,Fiat 500 1.2 69 Cv Lounge Navi,21445.0,Benzina,Manuale,9780.0,Lounge,2019,Lazio


<a id='mapping' />

## Mapping
[(back to top)](#content)

In [27]:
#mesi=[['Gennaio ','01-01-'], ['Febbraio ','01-02-'],['Marzo ','01-03-'], ['Aprile ','01-04-'],
#      ['Maggio ','01-05-'], ['Giugno ','01-06-'], ['Luglio ','01-07-'], ['Agosto ','01-08-'],
#      ['Settembre ','01-09-'], ['Ottobre ','01-10-'], ['Novembre ','01-11-'], ['Dicembre','01-12-']]
#
#for i,j in mesi: 
#    fiat_500['Month_Year'] = fiat_500['Month_Year'].str.replace(i,j)
#
#pd.to_datetime(fiat_500['Month_Year'], format='%d-%m-%Y', errors='coerce')